# Vector Indexes

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),
)

client.is_ready()

## Collection with HNSW index (default)

[HNSW params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#hnsw-index-parameters)

In [ ]:
from weaviate.classes.config import Configure, VectorDistances

client.collections.delete("IndexExample")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="IndexExample",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="main_vector",

            model="text-embedding-3-small",
            source_properties=['title', 'text'],

            # HSNW example 
            vector_index_config=Configure.VectorIndex.hnsw()
        )
    ],
)

## Collection with Flat index

[Flat params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#flat-indexes)

In [ ]:
from weaviate.classes.config import Configure, VectorDistances

client.collections.delete("IndexExample")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="IndexExample",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="main_vector",

            model="text-embedding-3-small",
            source_properties=['title', 'text'],

            # Flat example
            vector_index_config=Configure.VectorIndex.flat(
                # distance_metric=VectorDistances.COSINE, # optional
                vector_cache_max_objects=100_000,
            ),
        ),
    ],
)

## Collection with Dynamic index

[Dynamic params](https://weaviate.io/developers/weaviate/config-refs/schema/vector-index#dynamic-index-parameters)

In [ ]:
from weaviate.classes.config import Configure

client.collections.delete("IndexExample")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="IndexExample",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="main_vector",

            model="text-embedding-3-small",
            source_properties=['title', 'text'],

            # Dynamic example
            vector_index_config=Configure.VectorIndex.dynamic(
                threshold=10_000, # when to switch to HNSW

                flat=Configure.VectorIndex.flat(
                    vector_cache_max_objects=100_000,
                    # note: can also include a quantizer
                    quantizer=Configure.VectorIndex.Quantizer.bq()
                ),

                hnsw=Configure.VectorIndex.hnsw(
                    max_connections=32, # optional
                    # note: the quantizer can be different between flat and hnsw
                    quantizer=Configure.VectorIndex.Quantizer.pq()
                ),
            ),
        )
    ],
)

## The rest is the same

In [ ]:
# comment this out if you want to import 25k objects to your collection to test it
# from data_loader import import_wiki_data
# import_wiki_data(client, "IndexExample", 25_000)

In [ ]:
# index_example = client.collections.get("IndexExample")
# index_example.aggregate.over_all()

## Clean up

In [ ]:
client.collections.delete("IndexExample")

## Close the client

In [ ]:
client.close()